<a href="https://colab.research.google.com/github/pavelpryadokhin/Data-Scientist/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B0%D0%BA%D1%86%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ЦЕЛЬ

Используя датасет о стоимости акций [Сбербанка с 01.01.2013 года](https://storage.yandexcloud.net/academy.ai/SBER.csv)

Рассмотрим различные индикаторы

In [2]:
# Загрузим все необходимые библиотеки
import os
import numpy as np
import pandas as pd

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [3]:
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv

--2025-02-01 16:48:53--  https://storage.yandexcloud.net/academy.ai/SBER.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180264 (176K) [text/csv]
Saving to: ‘SBER.csv’

SBER.csv            100%[===================>] 176.04K   174KB/s    in 1.0s    

2025-02-01 16:48:56 (174 KB/s) - ‘SBER.csv’ saved [180264/180264]



In [4]:
df = pd.read_csv("./SBER.csv", sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    2718 non-null   int64  
 1   OPEN    2718 non-null   float64
 2   HIGH    2718 non-null   float64
 3   LOW     2718 non-null   float64
 4   CLOSE   2718 non-null   float64
 5   VOL     2718 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 127.5 KB


In [5]:
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y%m%d', errors='coerce').dt.date

df.index = range(len(df))
df.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,2013-01-08,96.50,98.50,96.12,98.37,92329970
1,2013-01-09,98.41,98.65,97.81,98.23,59776760
2,2013-01-10,98.35,98.55,97.96,98.41,47627940
3,2013-01-11,98.80,99.76,98.48,99.56,71837780
4,2013-01-14,99.72,101.27,99.57,100.99,83853170


#Полосы Боллинджера

In [6]:
fig = go.Figure([go.Scatter(x=df['DATE'], y=df['CLOSE'])])

fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [7]:
ma_size=20
df['SMA_20'] = df['CLOSE'].rolling(ma_size).mean()

bol_size=2
df['BB_UP'] = pd.Series(df['SMA_20']+ df['CLOSE'].rolling(ma_size).std() * bol_size)
df['BB_DOWN'] = pd.Series(df['SMA_20']- df['CLOSE'].rolling(ma_size).std() * bol_size)

In [8]:
fig = go.Figure()
fig.add_scatter(x=df.DATE, y=df.SMA_20, name='SMA 20')
fig.add_scatter(x=df.DATE, y=df.BB_UP, name='BB_UP')
fig.add_scatter(x=df.DATE, y=df.BB_DOWN, name='BB_DOWN')
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='CLOSE',opacity=0.3,line_color='black' )
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

##Способ прогнозирования

* Если цена закрытия пересекает нижнюю линию, это может указывать на возможное
изменение тренда на **восходящий**.

* Если цена закрытия пересекает верхнюю линию, есть вероятность, что тренд изменится на **нисходящий.**

# SMA   EMA RSI

Рассчитаем индикаторы:

In [9]:
# SMA - расчет скользящих средних
df['SMA_5'] = df['CLOSE'].rolling(5).mean().shift()
df['SMA_10'] = df['CLOSE'].rolling(10).mean().shift()
df['SMA_15'] = df['CLOSE'].rolling(15).mean().shift()
df['SMA_30'] = df['CLOSE'].rolling(30).mean().shift()

# EMA - расчет экспоненциальной скользящей средней
df['EMA_9'] = df['CLOSE'].ewm(span=9).mean().shift()
df['EMA_22'] = df['CLOSE'].ewm(span=22).mean().shift()

# RSI14 - индекс относительной силы на 14 дней
def RSI(df, n=14):
    close = df['CLOSE']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

df['RSI'] = RSI(df).fillna(0)

EMA_12 = pd.Series(df['CLOSE'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['CLOSE'].ewm(span=26, min_periods=26).mean())

# Расчет MACD и сигнала к сделке
df['MACD'] = pd.Series(EMA_12 - EMA_26)
df['MACD_signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())

Построим на одном графике скользящие средние за 5, 10, 15 и 30 дней:

In [12]:
fig = go.Figure()
fig.add_scatter(x=df.DATE, y=df.SMA_5, name='SMA 5')
fig.add_scatter(x=df.DATE, y=df.SMA_10, name='SMA 10')
fig.add_scatter(x=df.DATE, y=df.SMA_15, name='SMA 15')
fig.add_scatter(x=df.DATE, y=df.SMA_30, name='SMA 30')
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='Close', opacity=0.3)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Построим экспоненциальные скользящие средние за 9 дней и 22 на одном графике с ценой закрытия:

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.DATE, y=df.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=df.DATE, y=df.EMA_22, name='EMA 22'))
fig.add_trace(go.Scatter(x=df.DATE, y=df.CLOSE, name='Close', opacity=0.3))
fig.update_xaxes(rangeslider_visible=True)
fig.show()


1. Когда цена закрытия растет - EMA9 > EMA22 (так называемый "бычий тренд")
2. Когда цена закрытия падает - EMA9 < EMA22 (так называемый "медвежий тренд")
3. Пересечение EMA9 и EMA22 указывает на изменение тренда

В итоге мы получаем готовую торговую стратегию: когда EMA9 пересекает EMA22 снизу - сигнал к покупке, когда сверху - сигнал к продаже.

Аналогичный подход используется и для простых скользящих средних, а также их комбинаций.

Построим график индекса относительной силы (RSI):

In [14]:
fig = make_subplots(rows=3, cols=1)

fig.add_scatter(x=df.DATE, y=df.CLOSE, name='Close', row=1, col=1)
fig.add_scatter(x=df.DATE, y=df.RSI, name='RSI', row=2, col=1)
fig.add_scatter(x=df.DATE, y=df.VOL, name='Volume', row=3, col=1)

fig.show()

###Способ прогнозирования

**Индекс относительной силы (RSI):**

- **Общее описание**:  
  RSI — индикатор, проецирующий ценовые движения на шкалу от 0 до 100, позволяющий визуализировать истинную ценность изменений цен.

- **Совпадение направлений**:  
  Направление RSI совпадает с ценовым движением, что позволяет анализировать линии тренда и уровни поддержки/сопротивления на обоих графиках.

- **Линия поддержки на RSI**:  
  Если RSI на уровне 65, это указывает на устойчивый восходящий тренд (соотношение 65:35 между движением вверх и вниз).



---
**Сигналы для сделок:**

1. **Пересечение уровня 50**:
   - *Снизу вверх*: сигнал покупки (начало восходящего тренда).
   - *Сверху вниз*: сигнал продажи (начало нисходящего тренда).

2. **Дивергенция**:
   - *Медвежья дивергенция*: цена достигает новых максимумов, тогда как RSI ниже предыдущих максимумов (сигнал о возможном развороте вниз).
   - *Бычья дивергенция*: цена достигает новых минимумов, а RSI выше предыдущих минимумов (сигнал о возможном развороте вверх).

3. **Перекупленность и перепроданность**:
   - *RSI > 70*: перекупленность (ожидание продаж).
   - *RSI < 30*: перепроданность (ожидание покупок).

4. **Линии тренда на RSI**:  
   Использование линий тренда и уровней сопротивления на графике RSI для получения более точных сигналов.

#MACD

Построим график MACD и его сигнальной линии:

In [15]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='Close', row=1, col=1)
fig.add_scatter(x=df.DATE, y=EMA_12, name='EMA 12', row=1, col=1)
fig.add_scatter(x=df.DATE, y=EMA_26, name='EMA 26', row=1, col=1)
fig.add_scatter(x=df.DATE, y=df['MACD'], name='MACD', row=2, col=1)
fig.add_scatter(x=df.DATE, y=df['MACD_signal'], name='Signal line', row=2, col=1)
fig.show()

##Способ прогнозирования

**Сигнальная линия MACD:**

- **Определение**:  
  Сигнальная линия MACD — это экспоненциальная скользящая средняя основной линии MACD.

- **Пересечение с основной линией**:  
  Пересечение основной линии с сигнальной указывает на изменение тренда.

- **Сигналы о тренде**:
  - *Нисходящий тренд*:  
    Основная линия выше нуля пересекает сигнальную сверху вниз (возможный сигнал о начале нисходящего тренда).
  
  - *Восходящий тренд*:  
    Основная линия ниже нуля пересекает сигнальную снизу вверх (возможный сигнал о начале восходящего тренда).